In [1]:
import dask.dataframe as dd

In [2]:
import pandas as pd

In [3]:
import time

In [4]:
from math import sin, cos, sqrt, atan2, radians

In [5]:
colnames = ['tripduration', 'starttime', 'stoptime', 'startstationid',
            'startstationname', 'startlat', 'statlong',
            'endstationid', 'endstationname', 'endlat', 'endlong',
            'bikeid', 'usertype', 'year', 'gender']

In [6]:
path = r'/Users/eleanorelin/Documents/citi_bike/'

In [7]:
df43 = pd.read_csv(path + '201701-citibike-tripdata.csv')

In [8]:
df43.columns = colnames

In [9]:
df44 = pd.read_csv(path + '201702-citibike-tripdata.csv')
df44.columns = colnames
df45 = pd.read_csv(path + '201703-citibike-tripdata.csv')
df45.columns = colnames
df46 = pd.read_csv(path + '201704-citibike-tripdata.csv')
df46.columns = colnames
df47 = pd.read_csv(path + '201705-citibike-tripdata.csv')
df47.columns = colnames
df48 = pd.read_csv(path + '201706-citibike-tripdata.csv')
df48.columns = colnames
df49 = pd.read_csv(path + '201707-citibike-tripdata.csv')
df49.columns = colnames
df50 = pd.read_csv(path + '201708-citibike-tripdata.csv')
df50.columns = colnames
df51 = pd.read_csv(path + '201709-citibike-tripdata.csv')
df51.columns = colnames

In [10]:
frames = [df43,
          df44, df45, df46, df47, df48, df49, df50,
          df51]

In [11]:
result = pd.concat(frames)
result2 = result

In [12]:
len(result)

12246449

In [13]:
result.describe()

,tripduration,startstationid,startlat,statlong,endstationid,endlat,endlong,bikeid,year,gender
count,1.224645e+07,1.224645e+07,1.224645e+07,1.224645e+07,1.224645e+07,1.224645e+07,1.224645e+07,1.224645e+07,1.094050e+07,1.224645e+07
mean,1.010992e+03,1.322901e+03,4.073636e+01,-7.398509e+01,1.316123e+03,4.073603e+01,-7.398519e+01,2.317302e+04,1.978872e+03,1.116225e+00
std,1.470546e+04,1.329608e+03,3.831416e-02,5.023155e-02,1.327665e+03,5.432729e-02,8.626329e-02,4.970997e+03,1.174827e+01,5.686178e-01
min,6.100000e+01,7.200000e+01,0.000000e+00,-7.403137e+01,7.200000e+01,0.000000e+00,-7.408585e+01,1.452900e+04,1.858000e+03,0.000000e+00
25%,3.790000e+02,3.590000e+02,4.071782e+01,-7.399704e+01,3.580000e+02,4.071775e+01,-7.399724e+01,1.844200e+04,1.971000e+03,1.000000e+00
50%,6.350000e+02,4.840000e+02,4.073705e+01,-7.398759e+01,4.830000e+02,4.073705e+01,-7.398775e+01,2.516700e+04,1.982000e+03,1.000000e+00
75%,1.104000e+03,3.135000e+03,4.075514e+01,-7.397621e+01,3.132000e+03,4.075510e+01,-7.397634e+01,2.750900e+04,1.988000e+03,1.000000e+00
max,9.735948e+06,3.637000e+03,4.550636e+01,0.000000e+00,3.637000e+03,4.550636e+01,0.000000e+00,3.197900e+04,2.001000e+03,2.000000e+00


In [14]:
def calculate_dis(x):
    R = 6373.0
    lat1 = radians(x['startlat'])
    lon1 = radians(x['statlong'])
    lat2 = radians(x['endlat'])
    lon2 = radians(x['endlong'])
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R*c   
    return distance

In [15]:
start1 = time.time()

In [15]:
result['distance'] = result.apply(calculate_dis, axis = 1)

In [18]:
end1 = time.time()

In [19]:
end1 - start1 

1121.6598300933838

In [20]:
start2 = time.time()

In [16]:
ds = dd.from_pandas(result2, npartitions = 4)

In [17]:
ds['dis'] = ds.apply(calculate_dis, axis = 1)

/Users/eleanorelin/anaconda3/lib/python3.6/site-packages/dask/dataframe/core.py:2753: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


In [25]:
end2 = time.time()

In [26]:
end2 - start2

288.6736559867859

In [27]:
#return to pandas format

In [28]:
next(ds.iterrows())

(0, tripduration                                680
 starttime                   2017-01-01 00:00:21
 stoptime                    2017-01-01 00:11:41
 startstationid                             3226
 startstationname    W 82 St & Central Park West
 startlat                                40.7828
 statlong                               -73.9714
 endstationid                               3165
 endstationname      Central Park West & W 72 St
 endlat                                  40.7758
 endlong                                -73.9762
 bikeid                                    25542
 usertype                             Subscriber
 year                                       1965
 gender                                        2
 dis                                    0.874395
 Name: 0, dtype: object)

In [ ]:
start3 = time.time()

In [50]:
result.sort_values(by=['distance'])

,tripduration,starttime,stoptime,startstationid,startstationname,startlat,statlong,endstationid,endstationname,endlat,endlong,bikeid,usertype,year,gender,distance
425599,92,2017-08-08 17:46:13,2017-08-08 17:47:46,265,Stanton St & Chrystie St,40.722293,-73.991475,265,Stanton St & Chrystie St,40.722293,-73.991475,28126,Subscriber,1991.0,2,0.000000
218681,483,2017-03-08 13:18:18,2017-03-08 13:26:21,526,E 33 St & 5 Ave,40.747659,-73.984907,526,E 33 St & 5 Ave,40.747659,-73.984907,27314,Subscriber,1959.0,1,0.000000
562772,237,2017-05-12 00:07:57,2017-05-12 00:11:55,347,Greenwich St & W Houston St,40.728846,-74.008591,347,Greenwich St & W Houston St,40.728846,-74.008591,25348,Subscriber,2000.0,1,0.000000
1141622,777,2017-09-19 19:26:06,2017-09-19 19:39:04,3310,14 St & 7 Ave,40.663779,-73.983968,3310,14 St & 7 Ave,40.663779,-73.983968,31094,Subscriber,2001.0,1,0.000000
1543701,107,2017-07-28 18:08:48,2017-07-28 18:10:35,3461,Murray St & Greenwich St,40.714852,-74.011223,3461,Murray St & Greenwich St,40.714852,-74.011223,27389,Subscriber,1981.0,2,0.000000
430353,1758,2017-02-19 10:20:51,2017-02-19 10:50:10,276,Duane St & Greenwich St,40.717488,-74.010455,276,Duane St & Greenwich St,40.717488,-74.010455,16388,Subscriber,1981.0,1,0.000000
88891,344,2017-09-02 18:15:11,2017-09-02 18:20:56,537,Lexington Ave & E 24 St,40.740259,-73.984092,537,Lexington Ave & E 24 St,40.740259,-73.984092,25476,Subscriber,1958.0,1,0.000000
1141641,1685,2017-09-19 19:26:26,2017-09-19 19:54:32,217,Old Fulton St,40.702772,-73.993836,217,Old Fulton St,40.702772,-73.993836,30397,Customer,NaN,0,0.000000
51844,893,2017-02-02 14:09:39,2017-02-02 14:24:32,383,Greenwich Ave & Charles St,40.735238,-74.000271,383,Greenwich Ave & Charles St,40.735238,-74.000271,14702,Subscriber,1987.0,1,0.000000
339859,2079,2017-08-06 16:42:15,2017-08-06 17:16:55,322,Clinton St & Tillary St,40.696192,-73.991218,322,Clinton St & Tillary St,40.696192,-73.991218,19843,Customer,NaN,0,0.000000


In [ ]:
end3 = time.time()

In [30]:
start4 = time.time()

In [47]:
#sorting a parallel dataset requires expensive shuffles. Often we set_index once directly after data ingest and filtering and then perform many cheap computations off of the sorted dataset.

In [ ]:
#sorting in dask will change the partition structure

In [46]:
ds.set_index('tripduration') #cannot sort. too slow

KeyboardInterrupt: 

In [18]:
ds.dis.nlargest(2)

Dask Series Structure:
npartitions=1
    float64
        ...
Name: dis, dtype: float64
Dask Name: series-nlargest-agg, 21 tasks

In [22]:
result_filter = result[result.usertype == 'Subscriber']

In [21]:
ds_filter = ds[ds.usertype == 'Subscriber']

In [ ]:
end5 = time.time()

In [40]:
ds.groupby(ds.usertype).dis.mean()

Dask Series Structure:
npartitions=1
    float64
        ...
Name: dis, dtype: float64
Dask Name: truediv, 23 tasks

In [45]:
result.groupby('usertype')['distance'].mean()

usertype
Customer      2.041740
Subscriber    1.828849
Name: distance, dtype: float64

In [27]:
result.loc[:,'distance'] *= 0.6

KeyError: 'the label [distance] is not in the [index]'

In [29]:
ds['dis_miles'] = ds.loc[:, 'dis']*0.6

In [30]:
ds['dis_miles']

Dask Series Structure:
npartitions=4
0          float64
340180         ...
680359         ...
1156476        ...
1878097        ...
Name: dis_miles, dtype: float64
Dask Name: getitem, 28 tasks

In [31]:
ds.sum(axis=1)

Dask Series Structure:
npartitions=4
0          float64
340180         ...
680359         ...
1156476        ...
1878097        ...
dtype: float64
Dask Name: dataframe-sum, 28 tasks

In [34]:
result['distance'].sum()

13623170.680944411

In [35]:
ds['dis'].sum()

dd.Scalar<series-..., dtype=float64>

In [36]:
ds['dis'].sum().compute()

22705284.468193926

In [37]:
time.time()

1516583941.825572

In [38]:
ds['dis'].sum().compute()

22705284.468193926

In [39]:
#http://dask.pydata.org/en/latest/dataframe-performance.html